In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

In [2]:
pwd

'C:\\Users\\ygy91\\Desktop\\GX sales report'

In [3]:
# import data
histo_folder = ['2020 sales']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

temp = []
for i in month:
    temp.append(pd.read_csv(f'2020 sales//{i}.csv'))
df = pd.concat(temp).reset_index(drop = True)

In [4]:
# drop unneeded columns
col_drop = ['Warehouse Name', 'Must Ship By', 'Backorder Date', 'Shipping Account Number', 'Ship To Address', 'Ship To Address 2',
            'Ship To Zip', 'Ship To Phone', 'Inventory Send Date', 'Registered Timestamp', 'Customization Text', 'Event Name',
            'Event ID', 'Event Start Date', 'Event End Date', 'Event Type', 'Backorder Reason', 'Original Product ID', 
            'Original Product Name', 'Event Inventory Source', 'Packing Slip URL', 'Tracking Number', 'Ready for Pickup Date',
            'Destination Country', 'Depot ID', 'Depot Name', 'Wholesale Event Source', 'Wholesale Event Store Source', 
            'Composite Wood Product', 'Sales Channel']

In [5]:
df.drop(col_drop, axis = 1, inplace = True)

In [6]:
# Date Cleaning
date_col = ['PO Date', 'Must Ship By', 'Backorder Date']

df['PO Date'] = df.apply(lambda x: datetime.strptime(x['PO Date'],'%m/%d/%Y'), axis = 1)
df['Month'] = df.apply(lambda x: x['PO Date'].month, axis = 1)

In [7]:
# add category
# cate = pd.read_excel('sup source//category.xlsx', sheet_name = 'Category')
# df = df.merge(cate[['Item Name', 'Category']], on = 'Item Name', how = 'left')

In [8]:
# clean Item Number
df['Item Number'] = df.apply(lambda x: x['Item Number'][2:-1], axis = 1)

In [9]:
# clean PO Date & Time
df['PO Time'] = df['PO Date & Time'].str[11:13]

---

In [10]:
# process product info

df_pi = pd.read_csv('sup source//product info.csv')

In [11]:
df_pi['review & rating'] = df_pi.apply(lambda x: str(x['rating']).strip(str(x['Item Name'])), axis = 1)

In [12]:
p = re.compile(r'[(](.*?)[)]', re.S)

In [13]:
df_pi['Review'] = df_pi.apply(lambda x: 0
                        if re.findall(p, x['review & rating']) == [] 
                        else re.findall(p, x['review & rating'])[0], axis =1 )

In [14]:
df_pi['Rating'] = df_pi.apply(lambda x: x['review & rating'][x['review & rating'].index(':')+1:] if ':' in x['review & rating'] else 0, axis = 1)

In [15]:
df_pi['Date Launched'] = df_pi.apply(lambda x: datetime.strptime(x['Date Launched'],'%m/%d/%Y'), axis = 1)

In [16]:
df_pi = df_pi[['SKU','Date Launched','Satus','Review','Rating']].drop_duplicates()

In [17]:
df_pi

,SKU,Date Launched,Satus,Review,Rating
0,TNFI2298,2020-11-30,Live,0,0
1,TNFI2230,2020-10-02,Live,0,0
2,TNFI2240,2020-10-02,Live,0,0
3,TNFI2236,2020-10-02,Live,0,0
4,TNFI2235,2020-10-02,Live,0,0
...,...,...,...,...,...
790,TNFI1014,2018-03-28,Live,9,4.4
791,HAZE2464,2017-09-11,Live,8,4.9
792,HAZE2467,2017-09-11,Live,445,4.2
793,HAZE2468,2017-09-11,Live,1508,4.8


In [18]:
df=df.merge(df_pi, on ='SKU',how = 'left')

---

## Inventory

In [19]:
pwd

'C:\\Users\\ygy91\\Desktop\\GX sales report'

In [20]:
df_inv = pd.read_excel('inventory//20210104全球库存统计及补货计划表.xlsx', sheet_name = '入库商家补货计划表')

# data cleaning : 'Ⅰ'
df_inv['对应账号SKU'] = df_inv.apply(lambda x: str(x['对应账号SKU']).replace('Ⅰ','I'), axis =1 )

In [21]:
# clean 'Spare Parts'
df_inv = df_inv[df_inv['类别category'] != 'Spare Parts']

In [22]:
# 1. good columns

cols = ['国家', '对应账号SKU', '类别category',   '销量趋势(前30天对比（前60天到前30天）销量)',
        '第一次断货时间', '第二次断货时间', '要求ETD', '要求验货时间',
        '要求完工时间\n订舱\n调拨单', '要求下单时间', '建议下单数量', '货号', '入库商家',
       '机构', '四字机构', 'CBM', 'FOB出运港', '工厂']

warehouses = ['Cloud Stock\n(2019年04月01日至今)',
               'ComingSoon\n(2019年04月01日至今)', 'Ship Now\n(2019年04月01日至今)',
               'On the Way',
               'On the Way 2\n(Wayfair-Castle Gate仓\n US-Castle Gate仓\n CA - Castle Gate仓\n WAYFAIR-DE\n WAYFAIR-FR\n WAYFAIR-UK)',
               '仓1', '仓2', '仓3', 'Castle Gate仓', 'FBA仓',
               'FBC仓\n(对于MX，此列为FBM仓【Mercado Libre平台库存】)',
               '出口易库存仓1\n(对于US 美国，仓1为【美国库存】,\n对于CA 加拿大，仓1为【加拿大库存】)\n对于EU 欧洲，仓1为【英国库存】)',
               '出口易库存仓2\n(对于EU 欧洲，仓2为【法国库存】)', '出口易库存仓3\n(对于EU 欧洲，仓3为【德国库存】)', 'K2',
               '天池(FR LVA)', '兼爱(FR LBG)', '说剑(US SNA)', '美东一海通（天下）\n(US ATL)',
               '美东（天运）\n(US PHL)', '美西一海通（达生）\n(US BUR)', '美西（天道）\n(US LSQ)',
               '圆融汇通英国(让王)\n(UK MAN)', '圆融汇通（逍遥）\n(CA YTZ)', '德国嘉宏（山木）\n(DE DUS)',
               '大森林（宗师）\n(FR BVA)', 'Castle\n(OVERSTOCK)', 'CPA\n(OVERSTOCK)',
               'WKC\n(OVERSTOCK)', 'US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK)',
               '公冶(FR BOD)', 'Groupon仓\n(NL AMS)']

In [23]:
df_unpivot = df_inv.melt(id_vars = cols, value_vars = warehouses, var_name = 'warehouses')

In [24]:
map_WH  =   {'1. WH-Cloud': ['Cloud Stock\n(2019年04月01日至今)'],
             '2. WH-Coming': ["ComingSoon\n(2019年04月01日至今)"],
             '3. WH-ReadyToShip': ['Ship Now\n(2019年04月01日至今)'],
             '4. On the Way': ['On the Way', 'On the Way 2\n(Wayfair-Castle Gate仓\n US-Castle Gate仓\n CA - Castle Gate仓\n WAYFAIR-DE\n WAYFAIR-FR\n WAYFAIR-UK)'],
             '5. WH-Arrival': ['仓1', '仓2', '仓3', 'Castle Gate仓', 'FBA仓',
                     'FBC仓\n(对于MX，此列为FBM仓【Mercado Libre平台库存】)',
                     '出口易库存仓1\n(对于US 美国，仓1为【美国库存】,\n对于CA 加拿大，仓1为【加拿大库存】)\n对于EU 欧洲，仓1为【英国库存】)',
                     '出口易库存仓2\n(对于EU 欧洲，仓2为【法国库存】)', '出口易库存仓3\n(对于EU 欧洲，仓3为【德国库存】)',
                     'K2', '天池(FR LVA)', '兼爱(FR LBG)', '说剑(US SNA)',
                     '美东一海通（天下）\n(US ATL)', '美东（天运）\n(US PHL)', '美西一海通（达生）\n(US BUR)',
                     '美西（天道）\n(US LSQ)', '圆融汇通英国(让王)\n(UK MAN)', '圆融汇通（逍遥）\n(CA YTZ)',
                     '德国嘉宏（山木）\n(DE DUS)', '大森林（宗师）\n(FR BVA)', 'Castle\n(OVERSTOCK)',
                     'CPA\n(OVERSTOCK)', 'WKC\n(OVERSTOCK)',
                     'US-OVERSTOCK-CALIFORNIA\n(OVERSTOCK)', '公冶(FR BOD)',
                     'Groupon仓\n(NL AMS)']}

In [25]:
# select columns
df_unpivot.columns

Index(['国家', '对应账号SKU', '类别category', '销量趋势(前30天对比（前60天到前30天）销量)', '第一次断货时间',
       '第二次断货时间', '要求ETD', '要求验货时间', '要求完工时间\n订舱\n调拨单', '要求下单时间', '建议下单数量',
       '货号', '入库商家', '机构', '四字机构', 'CBM', 'FOB出运港', '工厂', 'warehouses',
       'value'],
      dtype='object')

In [26]:
df_wh = df_unpivot[['国家', '对应账号SKU', '类别category', '机构', '四字机构', 'CBM', 'FOB出运港', 
                    '入库商家', '工厂', 'warehouses', 'value']]

df_wh.columns = ['Country', 'Item Number', 'Category', 'Operation detail', 'Department-4', 
              'CBM', 'FOB-City', 'platform', 'Factory', 'warehouse detail', 'InvBal-Qty']

df_wh = df_wh[df_wh['InvBal-Qty'].notnull()].reset_index(drop=True)

In [27]:
map_WH.keys()

dict_keys(['1. WH-Cloud', '2. WH-Coming', '3. WH-ReadyToShip', '4. On the Way', '5. WH-Arrival'])

In [28]:
maps = []

for i in map_WH:
    temp = pd.DataFrame(map_WH[i], columns=['warehouse detail'])
    temp['Warehouse'] = i
    maps.append(temp)

maps_wh = pd.concat(maps).reset_index(drop = True)


In [29]:
maps_wh.head()

,warehouse detail,Warehouse
0,Cloud Stock\n(2019年04月01日至今),1. WH-Cloud
1,ComingSoon\n(2019年04月01日至今),2. WH-Coming
2,Ship Now\n(2019年04月01日至今),3. WH-ReadyToShip
3,On the Way,4. On the Way
4,On the Way 2\n(Wayfair-Castle Gate仓\n US-Castl...,4. On the Way


In [30]:
df_wh = df_wh.merge(maps_wh, on = 'warehouse detail', how = 'left')

In [31]:
df_wh

,Country,Item Number,Category,Operation detail,Department-4,CBM,FOB-City,platform,Factory,warehouse detail,InvBal-Qty,Warehouse
0,EU,ACER HM,Board with Metal Desk,梵蒂冈F1,协和广场,0.071,厦门,FDG,福建红梅家具有限公司,Cloud Stock\n(2019年04月01日至今),40.0,1. WH-Cloud
1,EU,ADOMO GLASS,Coffee Table Set,杨柳岸,协和广场,0.029,厦门,YLA_Willow_Fit@Home,福建红梅家具有限公司,Cloud Stock\n(2019年04月01日至今),600.0,1. WH-Cloud
2,EU,ADOMO WOOD,Coffee Table Set,杨柳岸,协和广场,0.029,厦门,YLA_Willow_Fit@Home,福建红梅家具有限公司,Cloud Stock\n(2019年04月01日至今),600.0,1. WH-Cloud
3,EU,ADRINA DOUBLE BLACK C,Double Bed,七泷津,渔人码头,0.048,南京,QLJ_Nanadaru_FURNITURER INC,安徽菲雷克斯科技发展有限公司,Cloud Stock\n(2019年04月01日至今),200.0,1. WH-Cloud
4,EU,ADRINA DOUBLE WHITE A,Double Bed,琅琊圃_FURNITURER INC,渔人码头,0.048,厦门,LYP_Lunar Pool_FURNITURER INC,福建红梅家具有限公司,Cloud Stock\n(2019年04月01日至今),500.0,1. WH-Cloud
...,...,...,...,...,...,...,...,...,...,...,...,...
23769,EU,MATAPOURI BLACK LT,Metal Locker,凯旋门_Furnish 1,木樨雨轩,0.128,青岛,KXM_GFBG_Furnish 1,洛阳莱特柜业（集团）有限公司,Groupon仓\n(NL AMS),3.0,5. WH-Arrival
23770,EU,MATAPOURI DARK GREY LT,Metal Locker,凯旋门_Furnish 1,木樨雨轩,0.128,青岛,KXM_GFBG_Furnish 1,洛阳莱特柜业（集团）有限公司,Groupon仓\n(NL AMS),16.0,5. WH-Arrival
23771,EU,MATAPOURI GREEN LT,Metal Locker,凯旋门_Furnish 1,木樨雨轩,0.128,青岛,KXM_GFBG_Furnish 1,洛阳莱特柜业（集团）有限公司,Groupon仓\n(NL AMS),16.0,5. WH-Arrival
23772,EU,MATAPOURI RED LT,Metal Locker,凯旋门_Furnish 1,木樨雨轩,0.128,青岛,KXM_GFBG_Furnish 1,洛阳莱特柜业（集团）有限公司,Groupon仓\n(NL AMS),13.0,5. WH-Arrival


In [32]:
################################   select only THTJ inventory   #################################

In [33]:
# df_wh.to_csv('PowerBI Input//Inventory Balances.csv', index=False)

---

## Product Catalog

In [34]:
# sales items
df_sales_item = df[['Item Number','SKU']]

# Upper
df_sales_item['Item Number'] = df_sales_item['Item Number'].str.upper()
df_sales_item['SKU'] = df_sales_item['SKU'].str.upper()
# clean '?' and strip
df_sales_item['Item Number'] = df_sales_item.apply(lambda x: str(x['Item Number']).replace('?',' ').strip(), axis =1)

df_sales_item = df_sales_item.drop_duplicates().reset_index(drop=True)

In [35]:
df_prod = pd.read_csv('sup source//Product Catalog//Product Catalog downloaded from Wayfair - 20201230.csv')

In [36]:
# product catalog items
df_rel = df_prod[['Supplier Part #', 'Internal SKU', 'Product Name']]

#Upper
df_rel['Supplier Part #'] = df_rel['Supplier Part #'].str.upper()
df_rel['Internal SKU'] = df_rel['Internal SKU'].str.upper()
df_rel['Product Name'] = df_rel['Product Name'].str.upper()
# clean '?' and strip
df_rel['Supplier Part #'] = df_rel.apply(lambda x: str(x['Supplier Part #']).replace('?',' ').strip(), axis =1)

df_rel = df_rel.drop_duplicates()
df_rel.rename(columns={'Supplier Part #':'Item Number', 'Internal SKU':'SKU'}, inplace=True)
df_rel = df_rel[df_rel['SKU'].notnull()].reset_index(drop=True)

In [37]:
# fuzzy items
df_fuzz = pd.read_csv('Item Mapping//fuzz_item.csv')
df_fuzz = df_fuzz[df_fuzz['ratio'] >= 90]
df_fuzz

,Item Number,fuzz_item,ratio,SKU
10,ALASKA OAK BLACK LEG,OAK,90,TNFI1420
19,ALDRIDGE GREY LMKZ,ALDRIDGE GREY,95,TNFI1631
24,ALDRIDGE VELVET CACTUS BROWN LEG,ALDRIDGE VELVET CACTUS OAK LEG,90,TNFI1797
29,AMABEL OAK WHITE,OAK,90,TNFI1420
32,AMAT WALNUT JM,AMAT WALNUT,95,TNFI1152
...,...,...,...,...
3268,BELSON OAK B,OAK,90,TNFI1420
3271,CHARLTON PP WHITE 2PCS,CHARLTON PP WHITE,95,TNFI1151
3275,DOREEN OAK,OAK,90,TNFI1420
3277,DUGAN BLUE I,DUGAN BLUE,95,TNFI1657


In [38]:
df_item = pd.concat([df_rel[['Item Number', 'SKU']],df_sales_item, df_fuzz[['Item Number', 'SKU']]]).drop_duplicates()

In [39]:
df_rel_name = df_rel[['SKU','Product Name']].drop_duplicates()

In [40]:
df_item = df_item.merge(df_rel_name, on='SKU', how='left')

In [41]:
# groupby Item Number ==> Make Sure item number are unique
df_item.sort_values(by = 'SKU', inplace = True)

In [42]:
df_item['SKU'] = ' && ' + df_item['SKU']
df_item['Product Name'] = ' && ' + df_item['Product Name']

In [43]:
df_item['SKU'] = df_item['SKU'].astype(str)
df_item['Product Name'] = df_item['Product Name'].astype(str)

In [44]:
df_item = df_item.groupby('Item Number').sum().reset_index()

In [45]:
df_item['SKU'] = df_item['SKU'].str[4:]
df_item['Product Name'] = df_item['Product Name'].str[4:]

In [46]:
df_item.fillna('Product Name not found in PC',inplace = True)

In [47]:
# finish processing sales and product catalog
# steps:
# 1. get unique Item Number and SKU (from both sales and product catalog and item_fuzzy)
# 2. put Product Name back from item_rel
# 3. sort and groupby

# below:
# 4. select sku from inventory but not in df_item
# 5. name 'unallocated' and concat in df_item
# 6. input category

In [48]:
# warehouse items
df_wh_item = df_wh[['Item Number']]

# upper
df_wh_item['Item Number'] =  df_wh_item['Item Number'].str.upper()
# clean '?' and strip
df_wh_item['Item Number'] = df_wh_item.apply(lambda x: str(x['Item Number']).replace('?',' ').strip(), axis =1)

df_wh_item = df_wh_item.drop_duplicates().reset_index(drop=True)

In [49]:
df_wh_item['in/out'] = df_wh_item.apply(lambda x: x['Item Number'] in list(df_item['Item Number']), axis = 1)

In [50]:
df_add = df_wh_item[~df_wh_item['in/out']][['Item Number']]
df_add['SKU'] = 'unallocated'      # define unallocated items

In [51]:
df_itemset = pd.concat([df_item, df_add]).fillna('unallocated inventory')    

In [52]:
# add category

In [53]:
category_info = df_prod[['Internal SKU', 'Wayfair Class']].drop_duplicates().reset_index(drop = True).rename(columns={'Internal SKU':'SKU - to match category', 'Wayfair Class':'Category'})

In [54]:
# add manual category_mapping input
df_man = pd.read_csv('Item Mapping//Items Mannual Input.csv')
df_man = df_man[['SKU', 'Category']].drop_duplicates().rename(columns={'SKU':'SKU - to match category'})

category_info = pd.concat([category_info,df_man])

In [55]:
df_itemset['SKU - to match category'] = df_itemset.apply(lambda x: str(x['SKU']).split(' && ')[0], axis = 1)

In [56]:
df_itemset = df_itemset.merge(category_info, on='SKU - to match category',how='left')
df_itemset = df_itemset.drop(columns = 'SKU - to match category')

In [57]:
# add Operation
df_teammap = pd.read_excel('sup source//team mapping//team mapping.xlsx', sheet_name='Sheet1').rename(columns={'Item Name':'Item Number'})


# Upper
df_teammap['Item Number'] = df_teammap['Item Number'].str.upper()
# clean '?' and strip
df_teammap['Item Number'] = df_teammap.apply(lambda x: str(x['Item Number']).replace('?',' ').strip(), axis =1)

temp = df_itemset.merge(df_teammap, on = 'Item Number', how = 'left')
# print(len(temp))
temp = temp[temp['Operation'].notnull()]
temp = temp[temp['SKU'] != 'unallocated']
temp = temp[['SKU', 'Operation']].drop_duplicates()

temp = temp.groupby('SKU').sum().reset_index()
temp

df_itemset = df_itemset.merge(temp, on = 'SKU', how = 'left')

In [58]:
df_itemset.to_csv('Item Mapping//DIM Item Name.csv', index=False, encoding='utf_8_sig')

In [59]:
df_itemset

,Item Number,SKU,Product Name,Category,Operation
0,3339-AM8583-BL,TNFI1025,KURLAND MESH OFFICE CHAIR,Office Chairs,NaN
1,3339-AM8585-GN,TNFI1025,KURLAND MESH OFFICE CHAIR,Office Chairs,NaN
2,3339-AM8587-00,TNFI1027,KUSHNER OFFICE CHAIR,Office Chairs,NaN
3,3339-AM8588-00,TNFI1001,IDYLBERRY DESK,Desks,NaN
4,3339-AM8589-00,TNFI1037,EVERBY TV STAND,TV Stands & Entertainment Centers,NaN
...,...,...,...,...,...
5036,YNEZ AQUA KD 2PC DT LMKZ,unallocated,unallocated inventory,NaN,NaN
5037,ZACHARY LIZY BEIGE,unallocated,unallocated inventory,NaN,NaN
5038,ZACHARY LIZY GREY I,unallocated,unallocated inventory,NaN,NaN
5039,ZACHARY LIZY PURPLE,unallocated,unallocated inventory,NaN,NaN


---

## Now save Inventory Fact Table (after finishing item mapping)

In [60]:
df_wh = df_wh.merge(df_itemset, on = 'Item Number', how = 'left')

In [61]:
df_wh.to_csv('PowerBI Input//Inventory Balances.csv', index=False)

## Now save Sales Fact Table (after finishing item mapping)

In [62]:
df.rename(columns={'SKU':'Wayfair SKU'}, inplace = True)

In [63]:
df = df.merge(df_itemset, on = 'Item Number', how = 'left')

In [64]:
df.to_csv('PowerBI Input//wayfair_sales_input.csv', index = False)

---

## fuzzy items

In [65]:
# from fuzzywuzzy import fuzz, process

# choices_sale = list(df_item['Item Number'])

# df_fuzz = df_add.apply(lambda x: process.extractOne(x['Item Number'], choices_sale), axis = 1)

# df_add['fuzz_item'] = [i[0] for i in df_fuzz]
# df_add['ratio'] = [i[1] for i in df_fuzz]

# df_pair = df_item.rename(columns={'Item Number':'fuzz_item'})

# df_add = df_add[['Item Number', 'fuzz_item', 'ratio']].merge(df_pair[['fuzz_item', 'SKU']], on = 'fuzz_item', how= 'left')

# df_add.to_csv('Item Mapping//fuzz_item.csv', index = False)

---

In [66]:
category_info = df_prod[['Supplier Part #', 'Wayfair Class']].drop_duplicates().reset_index(drop = True).rename(columns={'Supplier Part #':'Item Number', 'Wayfair Class':'Category'})

df_itemset.merge(category_info, on='Item Number',how='left')

,Item Number,SKU,Product Name,Category_x,Operation,Category_y
0,3339-AM8583-BL,TNFI1025,KURLAND MESH OFFICE CHAIR,Office Chairs,NaN,Office Chairs
1,3339-AM8585-GN,TNFI1025,KURLAND MESH OFFICE CHAIR,Office Chairs,NaN,Office Chairs
2,3339-AM8587-00,TNFI1027,KUSHNER OFFICE CHAIR,Office Chairs,NaN,Office Chairs
3,3339-AM8588-00,TNFI1001,IDYLBERRY DESK,Desks,NaN,Desks
4,3339-AM8589-00,TNFI1037,EVERBY TV STAND,TV Stands & Entertainment Centers,NaN,TV Stands & Entertainment Centers
...,...,...,...,...,...,...
5036,YNEZ AQUA KD 2PC DT LMKZ,unallocated,unallocated inventory,NaN,NaN,NaN
5037,ZACHARY LIZY BEIGE,unallocated,unallocated inventory,NaN,NaN,NaN
5038,ZACHARY LIZY GREY I,unallocated,unallocated inventory,NaN,NaN,NaN
5039,ZACHARY LIZY PURPLE,unallocated,unallocated inventory,NaN,NaN,NaN


In [67]:
a = df_itemset[df_itemset['SKU'] != 'unallocated']

In [68]:
a[a['Category'].isnull()]

,Item Number,SKU,Product Name,Category,Operation
1357,INDEPENDENCE COUNTER BAR BOUTON BLACK PU,MDRI1213 && TNFI1974,"&& LARES 27""COUNTER STOOL",NaN,初景滝
1899,PEAT,TNFI1353 && TNFI2239,&& ANDROMEDA 3 - PIECE DINING SET,NaN,初景滝
1900,PEAT HARDWARE,TNFI1353 && TNFI2239,,NaN,初景滝
1901,PEAT JM A,TNFI1353 && TNFI2239,,NaN,初景滝
1902,PEAT JM LMKZ,TNFI1353 && TNFI2239,,NaN,初景滝
1904,PEAT TARRANT,TNFI1353 && TNFI2239,,NaN,初景滝
2322,SMEG DARK BROWN PU BLACK GOLD LEG,TNFI1922,,NaN,初景滝
2328,SMEG GREEN PU BLACK GOLD LEG,TNFI1922,,NaN,初景滝
2329,SMEG GREY PU BLACK GOLD LEG,TNFI1922,,NaN,初景滝
2469,TRONCO BLUE,TNFI1198 && TNFI1944,&& RAFAELA VELVET TASK CHAIR,NaN,奈良井


In [69]:
df_itemset[df_itemset['SKU'] == 'TNFI1368']

,Item Number,SKU,Product Name,Category,Operation
584,CIAN BLUE,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
585,CIAN BLUE A,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
586,CIAN BLUE I,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
587,CIAN GREEN,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
588,CIAN GREEN A,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
589,CIAN GREEN I,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
590,CIAN GREY,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
591,CIAN GREY A,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
592,CIAN GREY I,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围
593,CIAN ROSE,TNFI1368,ROCHELLE TASK CHAIR,Office Chairs,沙田围


In [70]:
k = {'Item Number': ['SLIP GLASS Ⅰ', 'MCGHEE 246T PLUS', 'CIAN GREY Ⅰ', 'CIAN BLUE Ⅰ', 'CIAN GREEN Ⅰ', 
                     'RICO WHITE 6PCS RF'], 
     
     'SKU': ['TNFI1669', 'TNFI1795', 'TNFI1368', 'TNFI1368', 'TNFI1368', 'TNFI2232'], 
     
     'Product Name': ['FORRESTER DINING TABLE', 'TANVEER DESK', 'ROCHELLE TASK CHAIR', 'ROCHELLE TASK CHAIR', 'ROCHELLE TASK CHAIR',
                      'SHAMBLIN 7 - PIECE DINING SET']}

In [71]:
pd.DataFrame(k)

,Item Number,SKU,Product Name
0,SLIP GLASS Ⅰ,TNFI1669,FORRESTER DINING TABLE
1,MCGHEE 246T PLUS,TNFI1795,TANVEER DESK
2,CIAN GREY Ⅰ,TNFI1368,ROCHELLE TASK CHAIR
3,CIAN BLUE Ⅰ,TNFI1368,ROCHELLE TASK CHAIR
4,CIAN GREEN Ⅰ,TNFI1368,ROCHELLE TASK CHAIR
5,RICO WHITE 6PCS RF,TNFI2232,SHAMBLIN 7 - PIECE DINING SET


In [72]:
k = df_wh[df_wh['Country'] == 'CA']
k = k[k['Department-4'] == '太华天街']
k = k[k['Item Number'] == 'MCGHEE 246T PLUS'] 
k = k[k['Warehouse'] == '5. WH-Arrival']

k

,Country,Item Number,Category_x,Operation detail,Department-4,CBM,FOB-City,platform,Factory,warehouse detail,InvBal-Qty,Warehouse,SKU,Product Name,Category_y,Operation
13241,CA,MCGHEE 246T PLUS,Board with Metal Desk,曾厝垵_39FINC,太华天街,0.093,南京,ZCA_ANN_39FINC,安徽菲雷克斯科技发展有限公司,仓1,361.0,5. WH-Arrival,TNFI1795,TANVEER DESK,Desks,深水埗
13242,CA,MCGHEE 246T PLUS,Board with Metal Desk,青荇渡_39FINC,太华天街,0.093,南京,QXD_39FINC,安徽菲雷克斯科技发展有限公司,仓1,836.0,5. WH-Arrival,TNFI1795,TANVEER DESK,Desks,深水埗
13244,CA,MCGHEE 246T PLUS,Board with Metal Desk,鰂鱼涌,太华天街,0.093,南京,ZYC_AMAZON_39FINC,安徽菲雷克斯科技发展有限公司,仓1,79.0,5. WH-Arrival,TNFI1795,TANVEER DESK,Desks,深水埗
